<a href="https://colab.research.google.com/github/HsiuuYing/python-machine-learning/blob/main/%E5%B0%88%E9%A1%8C%E4%B8%89Skin_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 資料來源
Skin Cancer MNIST: HAM10000 https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000

### 資料準備

In [ ]:
#keras.utils: 做one-hot encoding用
#sklearn.model_selection: 分割訓練集和測試集
#os: 用來建立檔案、刪除檔案
#PIL: (圖像處理庫)匯入圖像
#seed: 設定種子，使每次隨機產生的資料有相同結果。可將數字改成自己的學號(或其他數字)
import numpy as np
import pandas as pd
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import os
from PIL import Image
np.random.seed(409570092)

In [ ]:
#7項皮膚疾病簡稱與全名
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
pd.Categorical(lesion_type_dict).codes

array([5, 4, 2, 1, 0, 6, 3], dtype=int8)

In [ ]:
!pip uninstall gdown -y && pip install gdown
!gdown -V

Found existing installation: gdown 4.6.0
Uninstalling gdown-4.6.0:
  Successfully uninstalled gdown-4.6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gdown-4.6.0-py3-none-any.whl (14 kB)
gdown 4.6.0 at /usr/local/lib/python3.8/dist-packages


In [ ]:
# Download from Google Drive
import gdown
url = 'https://drive.google.com/uc?id=1kklF0GDZ-4Vh52MIdTexky6Bqzek7S-c'
output = 'project03.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1kklF0GDZ-4Vh52MIdTexky6Bqzek7S-c
To: /content/project03.zip
100%|██████████| 26.6M/26.6M [00:00<00:00, 307MB/s]


'project03.zip'

In [ ]:
!unzip project03.zip

Archive:  project03.zip
replace project3_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: project3_test.csv       
  inflating: project3_train.csv      


In [ ]:
#讀取影像資料，28*28*3個像素值欄位(pixel0000-pixel2351) + 1個分類類別欄位label #img_rows, img_cols
load_img = pd.read_csv('project3_train.csv')

In [ ]:
#列出data的標籤
load_img.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,30,15,20,35,19,27,94,69,73,152,...,22,9,13,11,2,4,9,1,0,0
1,1,0,0,7,1,5,103,56,68,192,...,127,72,74,24,5,6,0,1,1,0
2,129,91,92,182,145,145,205,169,168,189,...,64,39,41,5,2,6,2,2,1,0
3,9,8,8,11,10,10,10,9,9,24,...,11,9,10,8,7,7,5,5,5,0
4,26,13,19,25,10,17,24,6,5,23,...,22,6,9,27,9,10,23,5,6,0


In [ ]:
#檢查讀取圖片的大小與數量
load_img.shape

(8008, 2353)

In [ ]:
load_img.iloc[: , :-1].values

array([[ 30,  15,  20, ...,   9,   1,   0],
       [  1,   0,   0, ...,   0,   1,   1],
       [129,  91,  92, ...,   2,   2,   1],
       ...,
       [127, 101, 108, ..., 121, 108, 125],
       [157,  82,  86, ..., 210, 126, 130],
       [176, 149, 166, ..., 175, 142, 159]])

In [ ]:
#iloc選取特定範圍，讀取種類編號
X_img , y_label = load_img.iloc[: , :-1].values , load_img.iloc[: , -1].values

In [ ]:
#將串列轉成矩陣
X_img_train = np.asarray(X_img.tolist())

#將一維的數據，轉換成三維(長*寬*RGB三色)
X_img_train=X_img_train.reshape(X_img_train.shape[0],28,28,3)

In [ ]:
#檢查學習資料的照片數量、尺寸大小、維度
print("train data:",'images:',X_img_train.shape," labels:",y_label.shape) 

train data: images: (8008, 28, 28, 3)  labels: (8008,)


In [ ]:
#標準化: 同除255(因為image的數字是0~255)
X_img_train_normalize = X_img_train.astype('float32') / 255.0

In [ ]:
#使用np_utils.to_categorical()傳入各參數的label標籤欄位，再執行OneHot encoding (轉成0或1的組合)
y_label_train_OneHot = np_utils.to_categorical(y_label)

In [ ]:
#檢查標籤總共有多少種分類
#這裡是共8008筆資料，每筆是7個0或1的組合
y_label_train_OneHot.shape

(8008, 7)

### 建立與訓練CNN模型

In [ ]:
#匯入keras中的Sequential、layers模組(Dense、 Dropout、 Activation、 Flatten、Conv2D、 MaxPooling2D、 ZeroPadding2D)
from keras.models import Sequential  #堆疊模型
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D #捲積、池化

In [ ]:
# 搭建CNN模型的第一層 只有第一層寫shape
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(3, 3),input_shape=(28, 28,3),activation='relu', padding='same'))
model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu', padding='same'))
# 池化
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# 第二層
model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# 第三層
model.add(Conv2D(filters=64, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(Conv2D(filters=64, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# 第四層
model.add(Conv2D(filters=128, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(Conv2D(filters=128, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# 全連接層的第一層 平坦層
model.add(Flatten())

In [ ]:
# 全連接層的第二層 隱藏層
model.add(Dense(64, activation='relu'))
# 避免overfitting
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

In [ ]:
# 最後一層為輸出層
model.add(Dense(7, activation='softmax'))

In [ ]:
# 打包網路模型
model.compile(loss='categorical_crossentropy',
       optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
# 訓練網路模型
# 開始訓練 (記得確認是否已設定使用GPU的運算資源)
train_history=model.fit(X_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.2,
                        epochs=15, batch_size=32, verbose=1)  

Epoch 1/15
201/201 [==============================] - 3s 10ms/step - loss: 0.8633 - categorical_accuracy: 0.7168 - val_loss: 4.4260 - val_categorical_accuracy: 0.3739
Epoch 2/15
201/201 [==============================] - 2s 12ms/step - loss: 0.6816 - categorical_accuracy: 0.7435 - val_loss: 7.1942 - val_categorical_accuracy: 0.3739
Epoch 3/15
201/201 [==============================] - 2s 11ms/step - loss: 0.6455 - categorical_accuracy: 0.7446 - val_loss: 11.6016 - val_categorical_accuracy: 0.3739
Epoch 4/15
201/201 [==============================] - 2s 12ms/step - loss: 0.6218 - categorical_accuracy: 0.7680 - val_loss: 10.0272 - val_categorical_accuracy: 0.3589
Epoch 5/15
201/201 [==============================] - 3s 12ms/step - loss: 0.6226 - categorical_accuracy: 0.7757 - val_loss: 10.2918 - val_categorical_accuracy: 0.3702
Epoch 6/15
201/201 [==============================] - 2s 12ms/step - loss: 0.6051 - categorical_accuracy: 0.7777 - val_loss: 9.0404 - val_categorical_accuracy: 0.

In [ ]:
# 使用最後的模型進行測試資料預測
load_test_img = pd.read_csv('project3_test.csv')
img_test = load_test_img.values

In [ ]:
x_test=img_test.reshape(img_test.shape[0],28,28,3)
x_test_normalize = x_test.astype('float32') / 255.0

輸出

In [ ]:
df_submit = pd.DataFrame([], columns=['Id', 'Label'])
df_submit['Id'] = [f'{i:04d}' for i in range(len(x_test_normalize))]
df_submit['Label'] = np.argmax(model.predict(x_test_normalize), axis=-1)

63/63 [==============================] - 0s 3ms/step


In [ ]:
df_submit.to_csv('submission_CNN11.csv', index=None)